In [13]:
import pandas as pd

In [14]:
df_taxes_list = []
path = "~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/"
for i in range(2009, 2020):
    df_taxes_list.append(pd.read_csv(path+"AY"+str(i)+ " Real Property_Personal Property.csv"))
df_full_taxes = pd.concat(df_taxes_list, ignore_index=True)
df_full_taxes.head

/anaconda3/envs/cds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/cds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
for (number, direction, name, type) in zip(df_['HOUSE_NO'], df['STDIRECT'], ETC):

In [25]:
df_full_taxes['HOUSE_NO'].value_counts().index

Int64Index([   0, 2601, 2734, 2512,  500,  509,  507,  206,  801,  407,
            ...
             163, 1060,  176,  162, 1259,  166, 1241, 4925, 4929, 6625],
           dtype='int64', length=3049)

In [23]:
df_full_taxes[df_full_taxes['HOUSE_NO']==0]

,OWNER_NAME,HOUSE_NO,STDIRECT,STREET_NAM,STREET_TYPE,UNIT,PROP_ZIP,ZONE_CODE,SUBDIVISION,PARCEL_NO,PREV_VAL,CURR_VAL,VALCHGDATE,TAXDISTRIC,HOMEEXEMPT,TOTALACRES,PROPERTY_CLASS,DIG_STRAT,BUSI_ID,NAICS,PERSKEY
0,NICHOLS DORRIS WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/014,13100,13100,11/12/2008,2,S0,0.00,RESIDENTIAL,3,NaN,NaN,NaN
1,WESTON MONTY,0,,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/015,26300,26300,11/12/2008,2,S0,0.39,COMMERCIAL,3,NaN,NaN,NaN
2,NICHOLS JOE,0,,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/016,990,990,11/12/2008,2,S0,0.25,COMMERCIAL,3,NaN,NaN,NaN
3,ISRAEL CAROLYN R ETAL,0,NaN,A C L RAILROAD,NaN,NaN,NaN,AG,MASSEY,00155/00004/013,21400,21400,4/16/2007,1,S0,17.19,RESIDENTIAL,4,NaN,NaN,NaN
4,GEER RET,0,NaN,A C L RAILROAD,NaN,NaN,NaN,AG,LIBERTY EXPRESSWAY INDUSTRIAL AREA,00186/00001/006,28600,28600,6/20/2007,1,S0,14.30,RESIDENTIAL,4,NaN,NaN,NaN
5,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/031,3200,3200,NaN,2,S0,0.21,RESIDENTIAL,3,NaN,NaN,NaN
6,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/030,4300,4300,NaN,2,S0,0.21,RESIDENTIAL,3,NaN,NaN,NaN
7,HENDERSON PEARL WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/004,3000,3000,3/6/2009,2,S0,0.17,RESIDENTIAL,3,NaN,NaN,NaN
8,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R6,COUNTY LINE - ACREE AREA,00104/00001/013,4000,4000,11/12/2008,2,S0,1.05,RESIDENTIAL,3,NaN,NaN,NaN
9,HENDERSON PEARL WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/003,9400,9400,10/31/2008,2,S0,0.22,RESIDENTIAL,3,NaN,NaN,NaN


In [40]:
df_full_taxes['STREET_TYPE'][0]

nan

In [41]:
pd.isna(direction)

NameError: name 'direction' is not defined

In [3]:
pd.options.display.max_columns = 22
pd.options.display.max_rows = 400

In [4]:
df_taxes = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/AY2009 Real Property_Personal Property.csv')

In [5]:
df_home = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_home_v02.csv')
df_cdbg = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_cdbg.csv')

In [6]:
df_utilities2012 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv')
df_utilities2013 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2013.csv')
df_utilities2014 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2014.csv')
df_utilities2015 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2015.csv')
df_utilities2016 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2016.csv')
df_utilities2017 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2017.csv')
df_utilities2018 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2018.csv')
df_utilities2019 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2019.csv')
df_utilities_list = [df_utilities2012,df_utilities2013,df_utilities2014,df_utilities2015,
                    df_utilities2016,df_utilities2017,df_utilities2018,df_utilities2019]
list1 = []
list2 = []
for df_u in df_utilities_list:
    list1.append(df_u['Premise Address'])
for series_u in list1:
    for row in series_u:
        list2.append(row)
s_utilities = pd.Series(pd.Series(list2).unique())

FileNotFoundError: [Errno 2] File b'/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv' does not exist: b'/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv'

In [9]:
def genStrip():
    return lambda addr: addr.strip()

In [10]:
df_taxes['FULL_ADDRESS'] = df_taxes['HOUSE_NO'].astype(str).apply(genStrip()) + \
' ' + df_taxes["STREET_NAM"].apply(genStrip()) + ' ' + df_taxes["STREET_TYPE"].astype(str).apply(genStrip())

In [11]:
s_taxes = pd.Series(df_taxes['FULL_ADDRESS'].unique())

In [9]:
df_house = pd.concat([df_home, df_cdbg], ignore_index = True)
df_house.drop('Unnamed: 0', axis = 1, inplace = True)
s_house = pd.Series(df_house['Address'].unique())

In [10]:
s_utilities.shape

(30714,)

In [11]:
s_taxes.shape

(35098,)

In [12]:
s_house.shape

(670,)

In [12]:
df_taxes.head()

,OWNER_NAME,HOUSE_NO,STDIRECT,STREET_NAM,STREET_TYPE,UNIT,PROP_ZIP,ZONE_CODE,SUBDIVISION,PARCEL_NO,PREV_VAL,CURR_VAL,VALCHGDATE,TAXDISTRIC,HOMEEXEMPT,TOTALACRES,PROPERTY_CLASS,DIG_STRAT,BUSI_ID,NAICS,PERSKEY,FULL_ADDRESS
0,NICHOLS DORRIS WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/014,13100,13100,11/12/2008,2,S0,0.00,RESIDENTIAL,3.0,NaN,NaN,NaN,0 A C L RAILROAD nan
1,WESTON MONTY,0,,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/015,26300,26300,11/12/2008,2,S0,0.39,COMMERCIAL,3.0,NaN,NaN,NaN,0 A C L RAILROAD nan
2,NICHOLS JOE,0,,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/016,990,990,11/12/2008,2,S0,0.25,COMMERCIAL,3.0,NaN,NaN,NaN,0 A C L RAILROAD nan
3,ISRAEL CAROLYN R ETAL,0,NaN,A C L RAILROAD,NaN,NaN,NaN,AG,MASSEY,00155/00004/013,21400,21400,4/16/2007,1,S0,17.19,RESIDENTIAL,4.0,NaN,NaN,NaN,0 A C L RAILROAD nan
4,GEER RET,0,NaN,A C L RAILROAD,NaN,NaN,NaN,AG,LIBERTY EXPRESSWAY INDUSTRIAL AREA,00186/00001/006,28600,28600,6/20/2007,1,S0,14.30,RESIDENTIAL,4.0,NaN,NaN,NaN,0 A C L RAILROAD nan


In [13]:
s_house

0                        1707 EDGERLY AVE
1                        703 DARTMOUTH LN
2                      1115 W WADDELL AVE
3                        313 FORRESTAL LN
4                            1125 JOEL DR
5                          1519 MALONE DR
6                          719 ANDOVER LN
7                     305 S SHADOWLAWN DR
8                           1662 SUNNY LN
9                         511 MOULTRIE RD
10                          1100 SWIFT ST
11                   1009 E ROOSEVELT AVE
12                     1118 W WHITNEY AVE
13                           431 EBONY LN
14                           529 EBONY LN
15                           535 EBONY LN
16                           539 EBONY LN
17                           545 EBONY LN
18                           559 EBONY LN
19                           563 EBONY LN
20                           531 EBONY LN
21                           537 EBONY LN
22                           543 EBONY LN
23                           555 E

In [14]:
s_taxes

0            0 A C L RAILROAD nan
1         4408 A C L RAILROAD nan
2                   2700 ABBEY LN
3                   2701 ABBEY LN
4                   2702 ABBEY LN
5                   2703 ABBEY LN
6                   2704 ABBEY LN
7                   2705 ABBEY LN
8                   2706 ABBEY LN
9                   2707 ABBEY LN
10                  2708 ABBEY LN
11                  2710 ABBEY LN
12                  2712 ABBEY LN
13                  2714 ABBEY LN
14                  2715 ABBEY LN
15                  2716 ABBEY LN
16                  2717 ABBEY LN
17                  2718 ABBEY LN
18                  2719 ABBEY LN
19                  0 ABERDEEN RD
20               1900 ABERDEEN RD
21               1909 ABERDEEN RD
22                  1111 ACKER DR
23                  1112 ACKER DR
24                  1113 ACKER DR
25                  1114 ACKER DR
26                  1115 ACKER DR
27                  1116 ACKER DR
28                  1118 ACKER DR
29            

#### copy in cleaning code for each home and cdbg, then concatenate, then run.

In [15]:
s_house[~s_house.isin(s_taxes)]

2                      1115 W WADDELL AVE
7                     305 S SHADOWLAWN DR
8                           1662 SUNNY LN
11                   1009 E ROOSEVELT AVE
12                     1118 W WHITNEY AVE
25                         1802 SHARON DR
32                         514 EUGEMAR DR
37                   2230 S WASHINGTON ST
38                    1006 S JEFFERSON ST
42                        1212 S DAVIS ST
50                      1004 AMSTERDAM LN
54                      2620 S MADISON ST
55                    1410 S CAROLINA AVE
56                    1411 S CAROLINA AVE
57                    1412 S CAROLINA AVE
58                    1413 S CAROLINA AVE
59                    1414 S CAROLINA AVE
60                    1415 S CAROLINA AVE
61                    1416 S CAROLINA AVE
67                      504 S MAGNOLIA ST
69                     1110 W WADDELL AVE
70                          1605 SUNNY LN
71                          1609 SUNNY LN
72                         1407 E 